In [2]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from evaluate import evaluator

/home/leontioso/anaconda3/envs/msc_thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-28 17:34:12.334093: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 17:34:12.334446: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 17:34:12.338191: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 17:34:12.655604: I tensorflow/core/platform/cpu_feature_guard.cc:182] This 

In [4]:
echr = load_dataset("ecthr_cases",  "violation-prediction")

In [5]:
train_dataset, val_dataset, test_dataset = echr['train'], echr['validation'], echr['test']

In [4]:
#model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")


In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [7]:
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {"text": "\n".join(examples["facts"])}) for dataset in [train_dataset, val_dataset, test_dataset]]

In [8]:
def encode(examples):
    return tokenizer( examples["text"],
                     truncation=True, 
                     padding=True)

In [8]:
train_dataset, val_dataset, test_dataset = [dataset.map(encode, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]

Map: 100%|██████████| 1000/1000 [00:00<00:00, 1342.76 examples/s]


In [9]:
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {'labels' :list(1 if examples['labels'][i] else 0 for i in range(len(examples['labels'])))}, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]

Map: 100%|██████████| 1000/1000 [00:00<00:00, 3407.72 examples/s]


In [10]:
train_dataset[1]

{'facts': ['9.  The applicant is the monarch of Liechtenstein, born in 1945 and living in Vaduz (Liechtenstein).',
  '10.  The applicant’s late father, the former monarch of Liechtenstein, had been the owner of the painting Szene an einem römischen Kalkofen (alias Der große Kalkofen) of Pieter van Laer, which had formed part of his family’s art collection since at least 1767. Until the end of the Second World War the painting had been in one of the family’s castles on the territory of the now Czech Republic.',
  '11.  In 1946 the former Czechoslovakia confiscated the property of the applicant’s father which was situated in its territory, including the painting in question, under Decree no. 12 on the “confiscation and accelerated allocation of agricultural property of German and Hungarian persons and of those having committed treason and acted as enemies of the Czech and Slovak people” (dekretu prezidenta republiky č. 12/1945 Sb. o konfiskaci a urychleném rozdělení majetku Němců, Mad’ar

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels)

In [14]:
id2label = {0: "NON_VIOLATED", 1: "VIOLATED"}
label2id = {"NON_VIOLATED": 0, "VIOLATED": 1}

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="distilbert_ecthr_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.264600,0.397875,0.851000
2,0.234000,0.371874,0.849000


TrainOutput(global_step=1126, training_loss=0.2457566862631437, metrics={'train_runtime': 4075.4664, 'train_samples_per_second': 4.417, 'train_steps_per_second': 0.276, 'total_flos': 2384413175808000.0, 'train_loss': 0.2457566862631437, 'epoch': 2.0})

In [13]:
task_evaluator = evaluator("text-classification")
results_dict = {}
for metric in ["accuracy", "precision", "recall", "f1"]:
    results = task_evaluator.compute(
        model_or_pipeline="./distilbert_ecthr_model/checkpoint-1126/",
        data=test_dataset,
        metric=metric,
        tokenizer=tokenizer,
        strategy="simple",
        random_state=0,
        input_column='text',
        label_column='labels',
        label_mapping={"NON_VIOLATED": 0.0, "VIOLATED": 1.0},
    )
    metric_name, value = list(results.items())[0]
    results_dict[metric_name] = value

In [14]:
results_dict

{'accuracy': 0.866,
 'precision': 0.8673366834170855,
 'recall': 0.9976878612716763,
 'f1': 0.9279569892473118}